In [1]:
!pip install wptools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 27.0 MB/s eta 0:00:00
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=46dd52869a7bb7b1e8b0013e85b4998521d3f057ee440934f3fdbb6413c9d2a7
  Stored in directory: /root/.cache/pip/wheels/f3/96/6d/a7eba8f80d31cbd188a2787b81514d82fc5ae6943c44777659
Successfully built html2text


In [2]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.2 MB/s eta 0:00:00


In [3]:
!pip install beautifulsoup4

## Mount Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd 'drive/MyDrive/Thesis-MSc/wikipedia-scraping/'
# %cd 'Thesis-MSc/wikipedia-scraping/'

/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping


In [6]:
import requests
import wptools
import re
import pandas as pd
import os
import csv
import json
from deep_translator import GoogleTranslator
import urllib3
from bs4 import BeautifulSoup
import string

In [7]:
page = wptools.page('Alexander_Haig')
page.get_parse()

en.wikipedia.org (parse) Alexander_Haig
en.wikipedia.org (imageinfo) File:General Alexander Meigs Haig, Jr.jpg
Alexander Haig (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:General A...
  infobox: <dict(53)> name, image, caption, office, order, preside...
  iwlinks: <list(2)> https://commons.wikimedia.org/wiki/Category:A...
  pageid: 80140
  parsetree: <str(98645)> <root><template><title>short description...
  requests: <list(2)> parse, imageinfo
  title: Alexander Haig
  wikibase: Q107957
  wikidata_url: https://www.wikidata.org/wiki/Q107957
  wikitext: <str(72681)> {{short description|American army general...
}


In [8]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "Alexander_Haig",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('Alexander Haig')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
1980 U.S. presidential election
1980 US presidential election
1980 murders of U.S. missionaries in El Salvador
1982 Lebanon War
1982 invasion of the Falkland Islands
1984 United States presidential election
1987 Libertarian National Convention
1988 Democratic National Convention
1988 Democratic Party presidential primaries
1988 Republican National Convention
1988 Republican Party presidential primaries
1988 United States House of Representatives elections
1988 United States Senate elections
1988 United States gubernatorial elections
1988 United States presidential election
1992 United States presidential election
1st Infantry Division (United States)
26th Infantry Regiment (United States)
60 Minutes II
AOL
Abel P. Upshur
Academy of Achievement
Adna Chaffee
Advice and consent
Air Force Distinguished Service Medal
Air Medal
Al Gore
Al Gore 1988 presidential campaign
Al Haig
Alexander Haig, Viscount Dawick
Alexander Haig (physician)
Alexander Hamilton
Alexander Macomb (general)
Alf

In [9]:
def get_arabic_name(english_phrase):
    # if we remove all numbers from a string and the string remains empty - then we won't translate that
    english_phrase_mod = ''.join([i for i in english_phrase if not i.isdigit()])
    english_phrase_mod = english_phrase_mod.translate(str.maketrans('', '', string.punctuation))
    english_phrase_mod = english_phrase_mod.strip()
    if english_phrase_mod == '':
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1
    else:
        english_phrase = english_phrase_mod

    if GoogleTranslator(source='auto', target='ar').translate(english_phrase) == english_phrase:
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1

    response = requests.get('http://en.wikipedia.org/wiki/{}'.format(english_phrase))
    if response.status_code != 200:
        print('Cannot find the link: {}'.format('http://en.wikipedia.org/wiki/{}'.format(english_phrase)))
        return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

    http = urllib3.PoolManager()
    url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
    response = http.request('GET', url)

    # get languages and links
    soup = BeautifulSoup(response.data)
    links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

    if links != []:
        for language, link in links:
            if language == 'ar':
                response = http.request('GET', link)
                soup = BeautifulSoup(response.data)

                arabic_phrase = soup.title.text
                arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
                return arabic_phrase, link

    return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def to_delete(s):
    # return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s
    return 'name' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'Current' in s or 'Formerly' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s or 'cite' in s or 'citation' in s or 'Cite' in s or 'http' in s

def get_list_items(s):
    # s = re.sub('{{.*?}}', '', s)
    # print(1, s)
    s = re.sub('\'', '', s)
#     print(2, s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     print(3, s)
    s = re.sub('\n----', '<br>', s)
#     print(4, s)
    s = re.sub('\n\*', '<br>', s)
#     print(5, s)
    s = re.sub('<br />', '<br>', s)
#     print(6, s)
    s = re.sub('<br/>', '<br>', s)
    s = re.sub('<Br/>', '<br>', s)
#     print(7, s)
    s = re.sub('</ref>|ref||</ref>', '', s)
#     print(8, s)
    s = re.sub('</ref>', '', s)
#     print(9, s)
    s = re.sub('ref', '', s)
#     print(10, s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     print(11, s)
    s = s.translate({ord('{'): None, ord('}'): None})
#     print(11, s)
#     if '<br>' in s:
    s = s.split("<br>")
#     print(12, s)
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
#     print(13, s)
    s = [remove_file_instances(e).strip() for e in s]
#     print(14, s)
    s = [e for e in s if not to_delete(e)]
#     print(15,  s)
    s = list(set(s)) # keep unique entries

    s = [e for e in s if not e.isdigit()] # remove entries that are just numbers
#     print('s is now: ', s)
    if len(s) > 1:
        new_s = []
        for e in s:
            if ',' in e:
                splitted_e = e.split(',')
                for ee in splitted_e:
                    new_s.append(ee)
            elif 'to' in e:
                splitted_e = e.split('to')
                for ee in splitted_e:
                    new_s.append(ee)
            else:
                new_s.append(e)
    #     print('s is now after: ', new_s)
        new_s = [e.strip() for e in new_s if e.strip() != '']
        return new_s
    else:
        return s


def jaccard(a, b):
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


In [10]:
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [11]:
df_political_party_en = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Founder(s)'])
df_political_party_ar = pd.DataFrame(columns=['الاسم','الإيديولوجيا', 'الموقف السياسي', 'المؤسس (المؤسسون)'])

In [12]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                pass
#                                 founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                print(page.data['infobox'][lt])
                                founders += ','.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    print(name)
                    if isinstance(name, list):
                        if len(name) > 1:
                            print(name)
                            jsims = []
                            for i, n in enumerate(name):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name = [name[jsims.index(max(jsims))]]
                            print('name after: ', name)
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'

                    df_political_party_en = pd.concat([df_political_party_en, pd.DataFrame({
                        'Name': [','.join(name if isinstance(name, list) else name)],
                        'Ideology': [','.join(ideology) if isinstance(ideology, list) else ideology],
                        'Political Position': [','.join(position) if isinstance(position, list) else position],
                        'Founder(s)': [founders],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('ideology: ', ideology)
                    print('position: ', position)



                    # get the arabic name, ideology, political position, and founders of the political party
#                     page_ar = wptools.page(p, lang='ar')
#                     page_ar.get_parse()
#                     name_ar = page_ar.data['title']

                    if name != '':
                        print('START')
                        name_ar = []
                        for n in name:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('name: ', n)
                                print('name ar: ', n_ar),
                                print('name ar link: ', n_ar_link)
                                name_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        name_ar = '-'

                    print('START')
                    ideology_ar = []
                    for ideo in ideology:
                        ideo_ar, ideo_ar_link = get_arabic_name(ideo)
                        if ideo_ar != -1:
                            print('ideology: ', ideo)
                            print('ideology ar: ', ideo_ar),
                            print('ideo ar link: ', ideo_ar_link)
                            ideology_ar.append(ideo_ar)
                        else:
                            continue
                    print('END')

                    if position != '-':
                        position_ar = []
                        print('START')
                        for pos in position:
                            pos_ar, pos_ar_link = get_arabic_name(pos)
                            if pos_ar != -1:
                                print('position: ', pos)
                                print('position ar: ', pos_ar),
                                print('position ar link: ', pos_ar_link)
                                position_ar.append(pos_ar)
                            else:
                                continue
                        print('END')
                    else:
                        position_ar = '-'

                    print('START')
                    founders_ar = []
                    for f in founders.split(','):
                        if f.strip() != '':
                            f_ar, f_ar_link = get_arabic_name(f)
                            if f_ar != -1:
                                print('founder: ', f)
                                print('founder ar: ', f_ar),
                                print('founder ar link: ', f_ar_link)
                                founders_ar.append(f_ar)
                            else:
                                continue
                    print('END')


                    df_political_party_ar = pd.concat([df_political_party_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar)],
                        'الإيديولوجيا': [','.join(ideology_ar)],
                        'الموقف السياسي': [','.join(position_ar)],
                        'المؤسس (المؤسسون)': [','.join(founders_ar)],
                    })], ignore_index=True)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1980 U.S. presidential election
en.wikipedia.org (parse) 1980 US presidential election
en.wikipedia.org (parse) 1980 murders of U.S. missionaries in El ...
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1982 invasion of the Falkland Islands
en.wikipedia.org (imageinfo) File:Operación Rosario-Soldados arge...
en.wikipedia.org (parse) 1984 United States presidential election
en.wikipedia.org (parse) 1987 Libertarian National Convention
en.wikipedia.org (imageinfo) File:Ron paul.jpg
en.wikipedia.org (parse) 1988 Democratic National Convention
en.wikipedia.org (imageinfo) File:DPDukakis1988.png
en.wikipedia.org (parse) 1988 Democratic Party presidential primaries
en.wikipedia.org (parse) 1988 Republican National Convention
en.wikipedia.org (imageinfo) File:RP1988.png
en.wikipedia.org (parse) 1988 Republican Party presidential primaries
en.wikipedia.org (parse) 1988 United States House of Representati...
en.wikipedia.org (parse) 1988 United State

[[Joe Biden]]
[[Kamala Harris]]
[[Chuck Schumer]]
[[Hakeem Jeffries]]
['Democratic Party']
-----------------
name:  ['Democratic Party']
founders:  Joe Biden,Kamala Harris,Chuck Schumer,Hakeem Jeffries,
ideology:  ['Centrism', 'Factions:', 'unbulleted list', '<!-- Concise list of factions below. -->', '<!-- Do not add socialism or democratic socialism as per multiple talk page discussions. -->', 'Liberalism', 'Progressivism']
position:  ['efn', 'Center-left efn']
START
name:  Democratic Party
name ar:  الحزب الديمقراطي (توضيح) 
name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D8%AF%D9%8A%D9%85%D9%82%D8%B1%D8%A7%D8%B7%D9%8A_(%D8%AA%D9%88%D8%B6%D9%8A%D8%AD)
END
START
ideology:  Centrism
ideology ar:  وسطية (سياسة) 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%88%D8%B3%D8%B7%D9%8A%D8%A9_(%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9)
ideology:  Factions:
ideology ar:  الفصائل (مترجمه)
ideo ar link:  -1
Cannot find the link: http://en.wikipedia.org/wiki/unbulle

en.wikipedia.org (parse) Denis McDonough
en.wikipedia.org (imageinfo) File:Secretary McDonough, official p...
en.wikipedia.org (parse) Dennis Reimer
en.wikipedia.org (imageinfo) File:Reimer chief of staff.JPEG
en.wikipedia.org (parse) Deputy National Security Advisor (United...
en.wikipedia.org (parse) Deputy Supreme Allied Commander Europe
en.wikipedia.org (imageinfo) File:Gen. Christopher G. Cavoli SACEUR.jpg
en.wikipedia.org (parse) Deputy United States Secretary of State
en.wikipedia.org (imageinfo) File:Kurt Campbell as Deputy Secreta...
en.wikipedia.org (parse) Dick Cheney
en.wikipedia.org (imageinfo) File:46 Dick Cheney 3x4.jpg
en.wikipedia.org (parse) Dick Gephardt
en.wikipedia.org (imageinfo) File:Dick Gephardt portrait.jpg
en.wikipedia.org (parse) Dick Thornburgh
en.wikipedia.org (imageinfo) File:Dick Thornburgh.jpg
en.wikipedia.org (parse) Dictatorships and Double Standards
en.wikipedia.org (parse) Director of Central Intelligence
en.wikipedia.org (imageinfo) File:Allen w du

[[David North (socialist)|David North]]
[[Joseph Kishore]]
Lawrence Porter
['Socialist Equality Party']
-----------------
name:  ['Socialist Equality Party']
founders:  David North,Joseph Kishore,Lawrence Porter,
ideology:  ['plainlist', 'Trotskyism', 'Communism', 'Revolutionary socialism']
position:  ['Far-left']
START
name:  Socialist Equality Party
name ar:  حزب المساواة الاشتراكية (مترجمه)
name ar link:  -1
END
START
Cannot find the link: http://en.wikipedia.org/wiki/plainlist
ideology:  plainlist
ideology ar:  قائمة عادية (مترجمه)
ideo ar link:  -1
ideology:  Trotskyism
ideology ar:  تروتسكية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%AA%D8%B1%D9%88%D8%AA%D8%B3%D9%83%D9%8A%D8%A9
ideology:  Communism
ideology ar:  شيوعية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%B4%D9%8A%D9%88%D8%B9%D9%8A%D8%A9
ideology:  Revolutionary socialism
ideology ar:  اشتراكية ثورية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D8%B4%D8%AA%D8%B1%D8%A7%D9%83%D9%8A%D8%A9_%D8%AB%D9%88%D8%

en.wikipedia.org (parse) Edwin Meese
en.wikipedia.org (imageinfo) File:Edwin Meese III Attorney Genera...
en.wikipedia.org (parse) El Salvador
en.wikipedia.org (parse) Elihu B. Washburne
en.wikipedia.org (imageinfo) File:Elihu B. Washburne seated - Bra...
en.wikipedia.org (parse) Elihu Root
en.wikipedia.org (imageinfo) File:Elihu Root, bw photo portrait, ...
en.wikipedia.org (parse) Elizabeth Dole
en.wikipedia.org (imageinfo) File:Elizabeth Dole official photo.jpg
en.wikipedia.org (parse) Eric Shinseki
en.wikipedia.org (imageinfo) File:Eric Shinseki official Veterans...
en.wikipedia.org (parse) Erskine Bowles
en.wikipedia.org (imageinfo) File:Erskine Bowles in 2010.jpg
en.wikipedia.org (parse) Eugene McCarthy
en.wikipedia.org (imageinfo) File:1964 Portrait Eugene McCarthy.jpg
en.wikipedia.org (parse) Events leading to the Falklands War
en.wikipedia.org (parse) Falklands War
en.wikipedia.org (parse) Foreign Policy Research Institute
en.wikipedia.org (imageinfo) File:Logo Foreign Policy 

...

en.wikipedia.org (parse) Vietnam
en.wikipedia.org (parse) Vietnam Campaign Medal
en.wikipedia.org (imageinfo) File:Vietnam Campaign Medal with 196...
en.wikipedia.org (parse) Vietnam Civil Actions Medal
en.wikipedia.org (imageinfo) File:Vietnam Civil Actions Medal.png
en.wikipedia.org (parse) Vietnam Cross of Gallantry
en.wikipedia.org (imageinfo) File:Gallantry Cross (Vietnam).jpg
en.wikipedia.org (parse) Vietnam Gallantry Cross
en.wikipedia.org (imageinfo) File:Gallantry Cross (Vietnam).jpg
en.wikipedia.org (parse) Vietnam Service Medal
en.wikipedia.org (imageinfo) File:Vietnam Service Medal, obverse.png
en.wikipedia.org (parse) Vietnam War
en.wikipedia.org (parse) Views of Lyndon LaRouche and the LaRouch...
en.wikipedia.org (parse) W. Marvin Watson
en.wikipedia.org (imageinfo) File:Portrait officiel de W. Marvin ...
en.wikipedia.org (parse) Wade H. Haislip
en.wikipedia.org (imageinfo) File:Wade Haislip.jpg
en.wikipedia.org (parse) Walter J. Stoessel Jr.
en.wikipedia.org (imageinfo) 

Larry Holmes
['Workers World Party']
-----------------
name:  ['Workers World Party']
founders:  Larry Holmes,
ideology:  ['Marxism–Leninism', 'Marcyism', 'Communism', 'ubl']
position:  ['Far-left']
START
name:  Workers World Party
name ar:  حزب العمال العالمي (مترجمه)
name ar link:  -1
END
START
ideology:  Marxism–Leninism
ideology ar:  ماركسية لينينية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%A7%D8%B1%D9%83%D8%B3%D9%8A%D8%A9_%D9%84%D9%8A%D9%86%D9%8A%D9%86%D9%8A%D8%A9
ideology:  Marcyism
ideology ar:  سام مارسي 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%B3%D8%A7%D9%85_%D9%85%D8%A7%D8%B1%D8%B3%D9%8A
ideology:  Communism
ideology ar:  شيوعية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%B4%D9%8A%D9%88%D8%B9%D9%8A%D8%A9
Cannot find the link: http://en.wikipedia.org/wiki/ubl
ideology:  ubl
ideology ar:  أوبل (مترجمه)
ideo ar link:  -1
END
START
Cannot find the link: http://en.wikipedia.org/wiki/Farleft
position:  Far-left
position ar:  أقصى اليسار (مترجمه)
positio

en.wikipedia.org (parse) World Affairs
en.wikipedia.org (parse) World Business Review
en.wikipedia.org (imageinfo) File:Wbr logo.gif
en.wikipedia.org (parse) World War II Victory Medal
en.wikipedia.org (imageinfo) File:WorldWar2VictoryMedalUS.JPG
en.wikipedia.org (parse) X Corps (United States)
en.wikipedia.org (imageinfo) File:US X Corps SSI.png
en.wikipedia.org (parse) Zbigniew Brzezinski


skipping Wikipedia:Link rot
skipping Wikipedia:Verifiability
skipping Template:1988 United States presidential election
skipping Template:Cite book
skipping Template:Cite encyclopedia
skipping Template:Marriage
skipping Template:Marriage/doc
skipping Template:Reagan cabinet
skipping Template:US Army Chiefs of Staff
skipping Template:United States Secretaries of State
skipping Template:White House Chiefs of Staff
skipping Template talk:1988 United States presidential election
skipping Template talk:Reagan cabinet
skipping Template talk:US Army Chiefs of Staff
skipping Template talk:United States Secretaries of State
skipping Template talk:White House Chiefs of Staff
skipping Help:Authority control
skipping Help:CS1 errors
skipping Help:IPA/English
skipping Category:Articles with dead external links from January 2022
skipping Category:CS1 maint: location missing publisher
skipping Category:Use mdy dates from March 2012
skipping Category:Wikipedia articles needing factual verification fro

en.wikipedia.org (imageinfo) File:Zbigniew Brzezinski, 1977.jpg


In [13]:
df_political_party_en

,Name,Ideology,Political Position,Founder(s)
0,Democratic Party,"Centrism,Factions:,unbulleted list,<!-- Concis...","efn,Center-left efn","Joe Biden,Kamala Harris,Chuck Schumer,Hakeem J..."
1,Socialist Equality Party,"plainlist,Trotskyism,Communism,Revolutionary s...",Far-left,"David North,Joseph Kishore,Lawrence Porter,"
2,Libertarian Party,"Paleolibertarianism,Deon,logical libertarianis...",-,",,"
3,New Alliance Party,"Left-wing nationalism,Left-wing populism,Minor...",Far-left,",,"
4,Populist Party,"White nationalism,White supremacy,Right-wing p...",Far-right,",,"
5,Prohibition Party,"Temperance,Social conservatism,Christian democ...","Syncretic,Fiscal: Center-left,Social: Center-r...",",,"
6,Red Army Faction,"Maoism,Anti-fascism,Plainlist,Revolutionary so...",Far-left,",,"
7,Republican Party,The Inclusion-Moderation Thesis: The U.S. Repu...,"right-wing,Sources for right-wing:,Center-righ...","Steve Scalise,Mike Johnson,Senate Minority Lea..."
8,Socialist Equality Party,"plainlist,Trotskyism,Communism,Revolutionary s...",Far-left,"David North,Joseph Kishore,Lawrence Porter,"
9,Socialist Party of the United States of America,"Anti-capitalism,Radical democracy,Democratic s...",Left-wing,"unbulleted list,unbulleted list,Greg Pason,Pat..."


In [14]:
df_political_party_ar

,الاسم,الإيديولوجيا,الموقف السياسي,المؤسس (المؤسسون)
0,الحزب الديمقراطي (توضيح),"وسطية (سياسة) ,الفصائل (مترجمه),قائمة غير منقو...","مادة (مترجمه),مركز يساري (مترجمه)","جو بايدن ,كامالا هاريس ,تشاك شومر ,حكيم جيفريز"
1,حزب المساواة الاشتراكية (مترجمه),"قائمة عادية (مترجمه),تروتسكية ,شيوعية ,اشتراكي...",أقصى اليسار (مترجمه),"ديفيد نورث (مترجمه),جوزيف كيشور (مترجمه),لوران..."
2,الحزب الليبرالي (مترجمه),"تحررية أصلية ,ديون (مترجمه),الليبرتارية المنطق...",-,
3,حزب التحالف الجديد (مترجمه),"القومية اليسارية (مترجمه),الشعبوية اليسارية (م...",أقصى اليسار (مترجمه),
4,الحزب الشعبوي (مترجمه),"قومية بيضاء ,سيادة البيض ,الشعبوية اليمينية (م...",اليمين المتطرف (مترجمه),
5,حزب المنع,"الاعتدال (مترجمه),محافظة اجتماعية ,ديمقراطية م...","توفيق بين الأديان ,الوسط المالي اليساري (مترجم...",
6,جماعة الجيش الأحمر,"ماوية ,معاداة الفاشية ,قائمة بسيطة (مترجمه),اش...",أقصى اليسار (مترجمه),
7,حزب جمهوري (توضيح),أطروحة الإدماج والاعتدال: الحزب الجمهوري الأمي...,"يمينية ,مصادر اليمين (مترجمه),يمين الوسط (مترج...","ستيف سكاليس ,مايك جونسون (سياسي) ,زعيم الأقلية..."
8,حزب المساواة الاشتراكية (مترجمه),"قائمة عادية (مترجمه),تروتسكية ,شيوعية ,اشتراكي...",أقصى اليسار (مترجمه),"ديفيد نورث (مترجمه),جوزيف كيشور (مترجمه),لوران..."
9,الحزب الاشتراكي للولايات المتحدة,"مناهضة الرأسمالية ,ديمقراطية راديكالية ,الاشتر...",يسارية,"قائمة غير منقوطة (مترجمه),قائمة غير منقوطة (مت..."


In [15]:
datasets_dir = 'datasets_updated/Alexander_Haig/'

In [16]:
mkdir(datasets_dir)
df_political_party_en.to_excel(os.path.join(datasets_dir, 'political_parties_en.xlsx'), index=False)
df_political_party_ar.to_excel(os.path.join(datasets_dir, 'political_parties_ar.xlsx'), index=False)

In [17]:
df_politician_en = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Religion'])
df_politician_ar = pd.DataFrame(columns=['الدين'       , 'الجنسيه'      , 'الحزب السياسي' , 'الاسم'])

In [18]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'

                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'

                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    print(name_inserted)
                    if isinstance(name_inserted, list):
                        if len(name_inserted) > 1:
                            print(name_inserted)
                            jsims = []
                            for i, n in enumerate(name_inserted):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name_inserted = [name_inserted[jsims.index(max(jsims))]][0]
                            print('name after: ', name_inserted)
                        else:
                            if name_inserted != []:
                                print('name_inserted before: {}'.format(name_inserted))
                                # name_inserted = [name_inserted]
                                # name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                                name_inserted = name_inserted[0]
                                print('name_inserted after: {}'.format(name_inserted))

                    df_politician_en = pd.concat([df_politician_en, pd.DataFrame({
                        'Name': [name_inserted],
                        'Political Party': [','.join(party) if isinstance(party, list) else party],
                        'Nationality': [','.join(nationality) if isinstance(nationality, list) else nationality],
                        'Religion': [','.join(religion) if isinstance(religion, list) else religion],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('party: ', party)
                    print('nationality: ', nationality)

                    # get name from wikipedia
                    # get political party from wikipedia
                    # translate nationality
                    # translate religion
                    name_ar, name_ar_link = get_arabic_name(p)
                    print('name ar: ', name_ar)

                    if party != '-':
                        print('START')
                        political_party_ar = []
                        for p in party:
                            p_ar, p_ar_link = get_arabic_name(p)
                            if p_ar != -1:
                                print('political party: ', p)
                                print('political party ar: ', p_ar),
                                print('political party ar link: ', p_ar_link)
                                political_party_ar.append(p_ar)
                            else:
                                continue
                        print('END')
                    else:
                        political_party_ar = '-'

                    if nationality != '-':
                        print('START')
                        nationality_ar = []
                        for n in nationality:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('nationality: ', n)
                                print('nationality ar: ', n_ar),
                                print('nationality ar link: ', n_ar_link)
                                nationality_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        nationality_ar = '-'

                    if religion != '-':
                        print('START')
                        religion_ar = []
                        for r in religion:
                            r_ar, r_ar_link = get_arabic_name(r)
                            if r_ar != -1:
                                print('religion: ', r)
                                print('religion ar: ', r_ar),
                                print('religion ar link: ', r_ar_link)
                                religion_ar.append(r_ar)
                            else:
                                continue
                        print('END')
                    else:
                        religion_ar = '-'

                    df_politician_ar = pd.concat([df_politician_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                        'الحزب السياسي': [','.join(political_party_ar) if isinstance(political_party_ar, list) else political_party_ar],
                        'الجنسيه': [','.join(nationality_ar) if isinstance(nationality_ar, list) else nationality_ar],
                        'الدين': [','.join(religion_ar) if isinstance(religion_ar, list) else religion_ar],
                    })], ignore_index=True)

                    print('name ar: ', name_ar)
                    print('religion ar: ', political_party_ar)
                    print('party ar: ', party)
                    print('nationality ar: ', nationality_ar)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1980 U.S. presidential election
en.wikipedia.org (parse) 1980 US presidential election
en.wikipedia.org (parse) 1980 murders of U.S. missionaries in El ...
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1982 invasion of the Falkland Islands
en.wikipedia.org (imageinfo) File:Operación Rosario-Soldados arge...
en.wikipedia.org (parse) 1984 United States presidential election
en.wikipedia.org (parse) 1987 Libertarian National Convention
en.wikipedia.org (imageinfo) File:Ron paul.jpg
en.wikipedia.org (parse) 1988 Democratic National Convention
en.wikipedia.org (imageinfo) File:DPDukakis1988.png
en.wikipedia.org (parse) 1988 Democratic Party presidential primaries
en.wikipedia.org (parse) 1988 Republican National Convention
en.wikipedia.org (imageinfo) File:RP1988.png
en.wikipedia.org (parse) 1988 Republican Party presidential primaries
en.wikipedia.org (parse) 1988 United States House of Representati...
en.wikipedia.org (parse) 1988 United State

['Abel Upshur']
name_inserted before: ['Abel Upshur']
name_inserted after: Abel Upshur
-----------------
name:  ['Abel Upshur']
native_name:  -
nickname:  -
religion:  -
party:  ['Whig']
nationality:  -
Cannot find the link: http://en.wikipedia.org/wiki/Abel P Upshur
name ar:  آبل بي أبشور (مترجمه)
START
political party:  Whig
political party ar:  اليمين (مترجمه)
political party ar link:  -1
END
name ar:  آبل بي أبشور (مترجمه)
religion ar:  ['اليمين (مترجمه)']
party ar:  ['Whig']
nationality ar:  -


en.wikipedia.org (parse) Academy of Achievement
en.wikipedia.org (imageinfo) File:Academy of Achievement building.jpg
en.wikipedia.org (parse) Adna Chaffee
en.wikipedia.org (imageinfo) File:Gen. Adna R. Chaffee, half-leng...


['Adna Chaffee']
name_inserted before: ['Adna Chaffee']
name_inserted after: Adna Chaffee
-----------------
name:  ['Adna Chaffee']
native_name:  -
nickname:  -
religion:  -
party:  -
nationality:  -
name ar:  عدنا تشافي 
name ar:  عدنا تشافي 
religion ar:  -
party ar:  -
nationality ar:  -


en.wikipedia.org (parse) Advice and consent
en.wikipedia.org (parse) Air Force Distinguished Service Medal
en.wikipedia.org (imageinfo) File:Air Force Distinguished Service...
en.wikipedia.org (parse) Air Medal
en.wikipedia.org (imageinfo) File:AirMed.gif
en.wikipedia.org (parse) Al Gore
en.wikipedia.org (imageinfo) File:Al Gore, Vice President of the ...


...

['Jerry Persons']
name_inserted before: ['Jerry Persons']
name_inserted after: Jerry Persons
-----------------
name:  ['Jerry Persons']
native_name:  -
nickname:  -
religion:  -
party:  ['Republican']
nationality:  -
name ar:  ويلتون بيرسونز (مترجمه)
START
political party:  Republican
political party ar:  جمهوري (مترجمه)
political party ar link:  -1
END
name ar:  ويلتون بيرسونز (مترجمه)
religion ar:  ['جمهوري (مترجمه)']
party ar:  ['Republican']
nationality ar:  -


en.wikipedia.org (parse) Winfield Scott
en.wikipedia.org (imageinfo) File:Winfield Scott by Fredricks, 18...


['Winfield Scott']
name_inserted before: ['Winfield Scott']
name_inserted after: Winfield Scott
-----------------
name:  ['Winfield Scott']
native_name:  -
nickname:  ['flatlist']
religion:  -
party:  ['Whig']
nationality:  -
name ar:  وينفيلد سكوت 
START
political party:  Whig
political party ar:  اليمين (مترجمه)
political party ar link:  -1
END
name ar:  وينفيلد سكوت 
religion ar:  ['اليمين (مترجمه)']
party ar:  ['Whig']
nationality ar:  -


en.wikipedia.org (parse) Workers World Party
en.wikipedia.org (parse) World Affairs
en.wikipedia.org (parse) World Business Review
en.wikipedia.org (imageinfo) File:Wbr logo.gif
en.wikipedia.org (parse) World War II Victory Medal
en.wikipedia.org (imageinfo) File:WorldWar2VictoryMedalUS.JPG
en.wikipedia.org (parse) X Corps (United States)
en.wikipedia.org (imageinfo) File:US X Corps SSI.png
en.wikipedia.org (parse) Zbigniew Brzezinski
en.wikipedia.org (imageinfo) File:Zbigniew Brzezinski, 1977.jpg


['Zbigniew Brzezinski']
name_inserted before: ['Zbigniew Brzezinski']
name_inserted after: Zbigniew Brzezinski
-----------------
name:  ['Zbigniew Brzezinski']
native_name:  -
nickname:  -
religion:  -
party:  ['Democratic']
nationality:  -
name ar:  زبغنيو بريجينسكي 
START
political party:  Democratic
political party ar:  ديمقراطي (مترجمه)
political party ar link:  -1
END
name ar:  زبغنيو بريجينسكي 
religion ar:  ['ديمقراطي (مترجمه)']
party ar:  ['Democratic']
nationality ar:  -
skipping Wikipedia:Link rot
skipping Wikipedia:Verifiability
skipping Template:1988 United States presidential election
skipping Template:Cite book
skipping Template:Cite encyclopedia
skipping Template:Marriage
skipping Template:Marriage/doc
skipping Template:Reagan cabinet
skipping Template:US Army Chiefs of Staff
skipping Template:United States Secretaries of State
skipping Template:White House Chiefs of Staff
skipping Template talk:1988 United States presidential election
skipping Template talk:Reagan cabin

In [19]:
df_politician_en

,Name,Political Party,Nationality,Religion
0,Abel Upshur,Whig,-,-
1,Adna Chaffee,-,-,-
2,Al Gore,Democratic,-,-
3,Al Haig,-,-,-
4,Alexander Haig,-,-,-
...,...,...,...,...
315,William Westmoreland,-,-,-
316,Williston B. Palmer,-,-,-
317,Jerry Persons,Republican,-,-
318,Winfield Scott,Whig,-,-


In [20]:
df_politician_ar

,الدين,الجنسيه,الحزب السياسي,الاسم
0,-,-,اليمين (مترجمه),آبل بي أبشور (مترجمه)
1,-,-,-,عدنا تشافي
2,-,-,ديمقراطي (مترجمه),آل جور
3,-,-,-,أل هايغ
4,-,-,-,ألكسندر هيج الطبيب (مترجمه)
...,...,...,...,...
315,-,-,-,وليام ويستمورلاند
316,-,-,-,ويليستون ب بالمر (مترجمه)
317,-,-,جمهوري (مترجمه),ويلتون بيرسونز (مترجمه)
318,-,-,اليمين (مترجمه),وينفيلد سكوت


In [21]:
mkdir(datasets_dir)
df_politician_en.to_excel(os.path.join(datasets_dir, 'politicians_en.xlsx'), index=False)
df_politician_ar.to_excel(os.path.join(datasets_dir, 'politicians_ar.xlsx'), index=False)

In [22]:
df_locations_en = pd.DataFrame(columns=['Name', 'Subdivision type', 'Subdivision name'])
df_locations_ar = pd.DataFrame(columns=['اسم التقسيم'         , 'نوع التقسيم' , 'الاسم'])

In [23]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'government_type' in page.data['infobox'] or 'subdivision_type' in page.data['infobox']:
                    print(page)

                    if 'government_type' in page.data['infobox']:
                        name = get_list_items(page.data['infobox']['common_name']) if 'common_name' in page.data['infobox'] else  page.data['infobox']['conventional_long_name'] if 'conventional_long_name' in page.data['infobox'] else '-'
                    else:
                        name = get_list_items(page.data['infobox']['official_name']) if 'official_name' in page.data['infobox'] else page.data['infobox']['name'] if 'name' in  page.data['infobox'] else '-'

                    subdivision_type = get_list_items(page.data['infobox']['subdivision_type']) if 'subdivision_type' in page.data['infobox'] else '-'
                    subdivision_name = get_list_items(page.data['infobox']['subdivision_name']) if 'subdivision_name' in page.data['infobox'] else '-'

                    name = ','.join(name) if isinstance(name, list) and len(name) >= 1 else name
                    if isEnglish(name):
                        pass
                    else:
                        name = p

                    df_locations_en = pd.concat([df_locations_en, pd.DataFrame({
                        'Name': [name],
                        'Subdivision type': [','.join(subdivision_type) if isinstance(subdivision_type, list) else subdivision_type],
                        'Subdivision name': [','.join(subdivision_name) if isinstance(subdivision_name, list) else subdivision_name]
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('subdivision_type: ', subdivision_type)
                    print('subdivision_name: ', subdivision_name)


                    if isinstance(name, list):
                        name = name[0]
                    else:
                        pass

                    if name != '-':
                        name_ar, name_ar_link = get_arabic_name(name)
                        print('name ar: ', name_ar)

                        if subdivision_type != '-':
                            print('START')
                            subdivision_type_ar = []
                            for st in subdivision_type:
                                st_ar, st_ar_link = get_arabic_name(st)
                                if st_ar != -1:
                                    print('subdivision_type: ', st)
                                    print('subdivision_type ar: ', st_ar),
                                    print('subdivision_type ar link: ', st_ar_link)
                                    subdivision_type_ar.append(st_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_type_ar = '-'


                        if subdivision_name != '-':
                            print('START')
                            subdivision_name_ar = []
                            for sn in subdivision_name:
                                sn_ar, sn_ar_link = get_arabic_name(sn)
                                if sn_ar != -1:
                                    print('subdivision_name: ', sn)
                                    print('subdivision_name ar: ', sn_ar),
                                    print('subdivision_name ar link: ', sn_ar_link)
                                    subdivision_name_ar.append(sn_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_name_ar = '-'

                        df_locations_ar = pd.concat([df_locations_ar, pd.DataFrame({
                            'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                            'نوع التقسيم': [','.join(subdivision_type_ar) if isinstance(subdivision_type_ar, list) else subdivision_type_ar],
                            'اسم التقسيم': [','.join(subdivision_name_ar) if isinstance(subdivision_name_ar, list) else subdivision_name_ar],
                        })], ignore_index=True)

                    else:
                        print('skipping {} because name is not found'.format(p))

        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1980 U.S. presidential election
en.wikipedia.org (parse) 1980 US presidential election
en.wikipedia.org (parse) 1980 murders of U.S. missionaries in El ...
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1982 invasion of the Falkland Islands
en.wikipedia.org (imageinfo) File:Operación Rosario-Soldados arge...
en.wikipedia.org (parse) 1984 United States presidential election
en.wikipedia.org (parse) 1987 Libertarian National Convention
en.wikipedia.org (imageinfo) File:Ron paul.jpg
en.wikipedia.org (parse) 1988 Democratic National Convention
en.wikipedia.org (imageinfo) File:DPDukakis1988.png
en.wikipedia.org (parse) 1988 Democratic Party presidential primaries
en.wikipedia.org (parse) 1988 Republican National Convention
en.wikipedia.org (imageinfo) File:RP1988.png
en.wikipedia.org (parse) 1988 Republican Party presidential primaries
en.wikipedia.org (parse) 1988 United States House of Representati...
en.wikipedia.org (parse) 1988 United State

-----------------
name:  Ardmore, Pennsylvania
subdivision_type:  ['Country']
subdivision_name:  ['United States']
Cannot find the link: http://en.wikipedia.org/wiki/Ardmore Pennsylvania
name ar:  أردمور بنسلفانيا (مترجمه)
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  United States
subdivision_name ar:  الولايات المتحدة 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%88%D9%84%D8%A7%D9%8A%D8%A7%D8%AA_%D8%A7%D9%84%D9%85%D8%AA%D8%AD%D8%AF%D8%A9
END


en.wikipedia.org (parse) Argentina


-----------------
name:  Argentina
subdivision_type:  -
subdivision_name:  -
name ar:  الأرجنتين 


en.wikipedia.org (parse) Arlington National Cemetery
en.wikipedia.org (imageinfo) File:Arlington National Cemetery Seal.png
en.wikipedia.org (parse) Army Distinguished Service Medal
en.wikipedia.org (parse) Army of Occupation Medal
en.wikipedia.org (imageinfo) File:Army of Occupation Medal.jpg
en.wikipedia.org (parse) Arthur E. Brown Jr.
en.wikipedia.org (imageinfo) File:Arthur E Brown.jpg
en.wikipedia.org (parse) Arthur St. Clair
en.wikipedia.org (imageinfo) File:ArthurStClairOfficialPortrait-r...
en.wikipedia.org (parse) Attempted assassination of Ronald Reagan
en.wikipedia.org (imageinfo) File:President Ronald Reagan moments...
en.wikipedia.org (parse) Award numeral
en.wikipedia.org (parse) Bachelor of Science
en.wikipedia.org (imageinfo) File:Cesar Roberto Merino Ortega Bac...
en.wikipedia.org (parse) Bainbridge Colby
en.wikipedia.org (imageinfo) File:Bainbridge Colby, bw photo port...
en.wikipedia.org (parse) Bala Cynwyd, Pennsylvania


...

en.wikipedia.org (parse) Vietnam Campaign Medal
en.wikipedia.org (imageinfo) File:Vietnam Campaign Medal with 196...
en.wikipedia.org (parse) Vietnam Civil Actions Medal
en.wikipedia.org (imageinfo) File:Vietnam Civil Actions Medal.png
en.wikipedia.org (parse) Vietnam Cross of Gallantry
en.wikipedia.org (imageinfo) File:Gallantry Cross (Vietnam).jpg
en.wikipedia.org (parse) Vietnam Gallantry Cross
en.wikipedia.org (imageinfo) File:Gallantry Cross (Vietnam).jpg
en.wikipedia.org (parse) Vietnam Service Medal
en.wikipedia.org (imageinfo) File:Vietnam Service Medal, obverse.png
en.wikipedia.org (parse) Vietnam War
en.wikipedia.org (parse) Views of Lyndon LaRouche and the LaRouch...
en.wikipedia.org (parse) W. Marvin Watson
en.wikipedia.org (imageinfo) File:Portrait officiel de W. Marvin ...
en.wikipedia.org (parse) Wade H. Haislip
en.wikipedia.org (imageinfo) File:Wade Haislip.jpg
en.wikipedia.org (parse) Walter J. Stoessel Jr.
en.wikipedia.org (imageinfo) File:Walter J. Stoessel, Jr.jpg
e

-----------------
name:  -
subdivision_type:  ['Country']
subdivision_name:  ['United States']
skipping Washington, D.C. because name is not found


en.wikipedia.org (parse) Washington Institute for Near East Policy
en.wikipedia.org (parse) Watergate scandal
en.wikipedia.org (parse) Wesley Clark
en.wikipedia.org (imageinfo) File:General Wesley Clark official p...
en.wikipedia.org (parse) White House
en.wikipedia.org (imageinfo) File:White House north and south sides.jpg
en.wikipedia.org (parse) White House Chief of Staff
en.wikipedia.org (imageinfo) File:Jeff Zients, WHCOS.jpg
en.wikipedia.org (parse) White House chief of staff
en.wikipedia.org (imageinfo) File:Jeff Zients, WHCOS.jpg
en.wikipedia.org (parse) Willa Kenoyer
en.wikipedia.org (parse) William A. Marra
en.wikipedia.org (parse) William Bennett
en.wikipedia.org (imageinfo) File:Bill Bennett by Gage Skidmore.jpg
en.wikipedia.org (parse) William French Smith
en.wikipedia.org (imageinfo) File:Portrait officiel de William Fr...
en.wikipedia.org (parse) William H. Seward
en.wikipedia.org (imageinfo) File:William H. Seward portrait - re...
en.wikipedia.org (parse) William H. Web

skipping Wikipedia:Link rot
skipping Wikipedia:Verifiability
skipping Template:1988 United States presidential election
skipping Template:Cite book
skipping Template:Cite encyclopedia
skipping Template:Marriage
skipping Template:Marriage/doc
skipping Template:Reagan cabinet
skipping Template:US Army Chiefs of Staff
skipping Template:United States Secretaries of State
skipping Template:White House Chiefs of Staff
skipping Template talk:1988 United States presidential election
skipping Template talk:Reagan cabinet
skipping Template talk:US Army Chiefs of Staff
skipping Template talk:United States Secretaries of State
skipping Template talk:White House Chiefs of Staff
skipping Help:Authority control
skipping Help:CS1 errors
skipping Help:IPA/English
skipping Category:Articles with dead external links from January 2022
skipping Category:CS1 maint: location missing publisher
skipping Category:Use mdy dates from March 2012
skipping Category:Wikipedia articles needing factual verification fro

en.wikipedia.org (imageinfo) File:Zbigniew Brzezinski, 1977.jpg


In [27]:
df_locations_en

,Name,Subdivision type,Subdivision name
0,"Ardmore, Pennsylvania",Country,United States
1,Argentina,-,-
2,"Bala Cynwyd, Pennsylvania",Country,United States
3,-,Country,United States
4,Chalatenango,Country,El Salvador
5,El Salvador,-,-
6,-,Country,Vietnam
7,Argentina,-,-
8,-,Country,"United States,Flagu"
9,-,Country,United States


In [28]:
df_locations_ar

,اسم التقسيم,نوع التقسيم,الاسم
0,الولايات المتحدة,بلد,أردمور بنسلفانيا (مترجمه)
1,-,-,الأرجنتين
2,الولايات المتحدة,بلد,بنسلفانيا هيثين بالا (مترجمه)
3,السلفادور,بلد,تشالاتينانغو (مترجمه)
4,-,-,السلفادور
5,-,-,الأرجنتين
6,-,-,فيتنام الجنوبية
7,-,-,الاتحاد السوفيتي
8,-,-,فيتنام


In [29]:
mkdir(datasets_dir)
df_locations_en.to_excel(os.path.join(datasets_dir, 'locations_en.xlsx'), index=False)
df_locations_ar.to_excel(os.path.join(datasets_dir, 'locations_ar.xlsx'), index=False)